<a href="https://colab.research.google.com/github/ZerXXX0/sales-prediction/blob/main/xgboostPAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install xgboost

In [7]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load data
train_df = pd.read_csv('https://raw.githubusercontent.com/ZerXXX0/sales-prediction/refs/heads/main/dataset/train_final.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/ZerXXX0/sales-prediction/refs/heads/main/dataset/test_final.csv')

# Columns to drop
drop_cols = ['Unnamed: 0', 'TransactionID', 'MemberID']

# Prepare training data
train_df_clean = train_df.drop(columns=drop_cols)
X = train_df_clean.drop(columns=['next_buy'])
y = train_df_clean['next_buy']

# Save column order
train_cols = X.columns.tolist()

# Impute and scale
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Train-val split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)


In [8]:
clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_val)
val_acc = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:01:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Validation Accuracy: 90.10%


In [9]:
# Preprocess test set
test_df_clean = test_df.drop(columns=drop_cols)
test_df_clean = test_df_clean[train_cols]  # reorder columns

X_test_imputed = imputer.transform(test_df_clean)
X_test_scaled = scaler.transform(X_test_imputed)

# Predict
test_preds = clf.predict(X_test_scaled)

# Prepare final submission
submission = pd.DataFrame({
    "MemberID": test_df["MemberID"],
    "next_buy": test_preds
})
submission.to_csv("xgboost_submission.csv", index=False)

In [10]:
submission_df = pd.read_csv("xgboost_submission.csv")
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21098 entries, 0 to 21097
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   MemberID  21098 non-null  object
 1   next_buy  21098 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 329.8+ KB


In [11]:
submit_df = pd.read_csv("https://raw.githubusercontent.com/ZerXXX0/sales-prediction/refs/heads/main/dataset/sample_submission.csv")
# First, create a new dataframe from submission_df with duplicate MemberIDs removed.
# We keep the 'last' entry for each member.
submission_df_unique = submission_df.drop_duplicates(subset=['MemberID'], keep='last')

# Now, create the lookup map from this de-duplicated dataframe.
# This will succeed because the 'MemberID' index is now unique.
next_buy_lookup = submission_df_unique.set_index('MemberID')['next_buy']

# Proceed with the map operation as before. This will now work correctly.
submit_df['next_buy'] = submit_df['MemberID'].map(next_buy_lookup)

submit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6381 entries, 0 to 6380
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   MemberID  6381 non-null   object
 1   next_buy  6381 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 99.8+ KB


In [12]:
submit_df.to_csv('submission_XGBOOST.csv', index=False)

In [13]:
submit_df.head()

,MemberID,next_buy
0,c2a630e3d0dc77dac0f63424a9ae1438,0
1,3ecf7484c08418953e967a20de37051b,0
2,97bbd6c99a862f20657d9b2b1c77b2c8,0
3,3ce072ff9c6f2f4b7c95dbc08324a24d,0
4,ab0b0de2a1c85a40b5d58644aef745c0,0
